In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
import PIL
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, datasets, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Flatten, BatchNormalization, LeakyReLU, GlobalAveragePooling2D, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.applications.xception import Xception

In [ ]:
image_array = []
label_array = []
# https://www.kaggle.com/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals
# download the dataset; copy paste the animal subfolder(folder containing 90 different animals)to the below path
image_folder = r"./images/animals_for_upload"

In [ ]:
for animal in os.listdir(image_folder):
    for i in range(len(os.listdir(image_folder + '/' + animal))):
        img = cv2.imread(image_folder + '/' + animal + '/' + os.listdir(image_folder + '/' + animal)[i])
        resized_img = cv2.resize(img,(224,224))
        resized_img = resized_img / 255.0
        image_array.append(resized_img)
        label_array.append(animal)

images = np.array(image_array,dtype = 'float32')

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(label_array)
class_names = le.classes_
labels = le.transform(label_array)

labels = np.array(labels, dtype = 'uint8')
labels = np.resize(labels, (len(labels),1))

In [ ]:
import joblib

joblib.dump(le, 'label_encoder.joblib')
label_encoder = joblib.load('label_encoder.joblib')

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.30, stratify = labels)

In [ ]:
xception_model = Sequential()

xception_base_model = Xception(include_top = False, weights = "imagenet", input_shape = (224, 224, 3))
print(f'Number of layers in Xception : {len(xception_base_model.layers)}')

for layer in xception_base_model.layers[:]:
    layer.trainable = False

for layer in xception_base_model.layers[90:]:
    layer.trainable = True

xception_model.add(xception_base_model)
xception_model.add(GlobalAveragePooling2D())
xception_model.add(Dense(units = 90, activation = 'softmax'))
xception_model.summary()

early_stopping = EarlyStopping( monitor = 'val_accuracy', mode = 'max', min_delta = 1e-5, patience = 20, restore_best_weights = True, verbose = 0)

xception_model.compile(optimizer = keras.optimizers.RMSprop(learning_rate = 0.0001) , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])
xception = xception_model.fit(train_images, train_labels, batch_size = 32, epochs = 5, callbacks = [early_stopping], validation_split = 0.3)

In [ ]:
xception_predictions = xception_model.predict(test_images)
xception_predictions = np.argmax(xception_predictions,axis = 1)

In [ ]:
randomImage = np.random.randint(5,np.shape(test_images)[0])

plt.imshow(test_images[randomImage])
plt.title("Xception Model's Predicted Animal : " + str(le.inverse_transform([xception_predictions[randomImage]])))
plt.show()

In [ ]:
plt.figure(figsize = (30,10))

plt.subplot(2,2,1)
i_train_acc = xception.history['accuracy']
i_val_acc = xception.history['val_accuracy']
i_epoch = [i for i in range(len(i_val_acc))]
plt.plot(i_epoch , i_train_acc , 'go-' , label = 'Training Accuracy')
plt.plot(i_epoch, i_val_acc , 'ro-' , label = 'Validation Accuracy')
plt.title('Training & Validation Accuracy for Xception Net')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

In [ ]:
xception_model.save(r"C:\Users\DK524UF\OneDrive - EY\Documents\EY Ripples\Forest Carbon Project\model")

In [ ]:
model = keras.models.load_model(r"C:\Users\DK524UF\OneDrive - EY\Documents\EY Ripples\Forest Carbon Project\model")